# Data Set

*The FER2013 dataset used, is a popular facial expression recognition dataset used for training and evaluating emotion recognition models.*

*Link for the dataset* : **https://www.kaggle.com/datasets/msambare/fer2013/**

# Detection Code

*Importing the required dependencies*

In [2]:
import cv2
from keras.models import model_from_json
import numpy as np

*Loading the required model json file*

In [4]:
json_file = open("model_architecture.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

model.load_weights("model_weights.h5")
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)


In [10]:
def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

camera = cv2.VideoCapture(0)
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

*Detecting the emotions via camera by extracting features of each frame and then passing on to the model*

In [11]:
while True:
    ret, frame = camera.read()
    if not ret:
        print("Failed to capture frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (p, q, r, s) in faces:
        face_roi = gray[q:q + s, p:p + r]
        cv2.rectangle(frame, (p, q), (p + r, q + s), (255, 0, 0), 2)
        face_roi = cv2.resize(face_roi, (48, 48))
        img = extract_features(face_roi)
        pred = model.predict(img)
        prediction_label = labels[pred.argmax()]
        cv2.putText(frame, prediction_label, (p - 10, q - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))

    cv2.imshow("Emotion Recognition", frame)

    key = cv2.waitKey(1)
    if key == ord('q'):  # Press 'q' to exit
        break

camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 105ms/step
